In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import pandas as pd 

from scipy import optimize
from scipy import integrate
from scipy.integrate import odeint

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
                 
import seaborn as sns
import requests
from bs4 import BeautifulSoup

sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

In [2]:
def getCovidData(category):
    if category == "confirmed":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    
    elif category == "recovered":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
        
    elif category == "deaths":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

    soup = BeautifulSoup(response.content, 'html.parser')
    html_table=soup.find('table')
    all_rows=html_table.find_all('tr')
    jh_data_list=[]
    for pos,rows in enumerate(all_rows):
        if pos==0:
            header_list = [each_col.get_text(strip=True) for each_col in rows.find_all('th')]
        else:
            col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
            jh_data_list.append(col_list)
    header_list.insert(0,'index')
    jh_data_df=pd.DataFrame(jh_data_list,index=None).dropna()
    jh_data_df.columns=header_list
    jh_data_df.drop(columns=['index','Lat','Long'],axis=1)
    # jh_data_df.head()
    time_idx=jh_data_df.columns[5:]
    
    df_plot = pd.DataFrame({'date':time_idx})

    country_list=jh_data_df['Country/Region']
    for each in country_list:
        df_plot[each]=np.array(jh_data_df[jh_data_df['Country/Region']==each].iloc[:,5::].sum(axis=0))

    time_idx=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date] # convert to datetime
    time_str=[each.strftime('%Y-%m-%d') for each in time_idx] # convert back to date ISO norm (str)

    df_plot['date']=time_idx
    df_plot[df_plot.columns[1:]] = df_plot[df_plot.columns[1:]].astype(int)
    
    return df_plot


In [3]:
df_confirmed = getCovidData("confirmed")
df_recovered = getCovidData("recovered")
df_deaths = getCovidData("deaths")

In [4]:
df_confirmed.to_csv('../data/processed/COVID_small_flat_table_confirmed.csv',sep=';',index=False)
df_recovered.to_csv('../data/processed/COVID_small_flat_table_recovered.csv',sep=';',index=False)
df_deaths.to_csv('../data/processed/COVID_small_flat_table_deaths.csv',sep=';',index=False)

In [5]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table_confirmed.csv',sep=';')  
df_analyse.sort_values('date',ascending=True).head()

,date,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0,3000010,...,0,0,0,0,2,0,0,0,0,0


## Simulative approach to calculate SIR curves

### Programming the Exposed and Dead Compartment

In [6]:
def deriv(y, t, N, beta, gamma, delta, alpha, rho):
    S, E, I, R, D = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - (1 - alpha) * gamma * I - alpha * rho * I
    dRdt = (1 - alpha) * gamma * I
    dDdt = alpha * rho * I
    return dSdt, dEdt, dIdt, dRdt, dDdt

In [7]:
N = 83832432
D = 5.0 # infections lasts five days
gamma = 1.0 / D
delta = 1.0 / 5.0  # incubation period of five days
R_0 = 5.0
beta = R_0 * gamma  # R_0 = beta / gamma, so beta = R_0 * gamma
alpha = 0.2  # 20% death rate
rho = 1/14  # 14 days from infection until death
S0, E0, I0, R0, D0 = N-1, 1, 0, 0, 0  # initial conditions: one exposed

In [8]:
t_de = np.linspace(0, 99, 100) # Grid of time points (in days)
y0 = S0, E0, I0, R0, D0 # Initial conditions vector

# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t_de, args=(N, beta, gamma, delta, alpha, rho))
S, E, I, R, D = ret.T

In [9]:
seird_dataframe = pd.DataFrame({'susceptible':S,
                                        'exposed' : E,
                                        'infected': I,
                                        'recovered': R,
                                         'dead': D
                                         })

seird_dataframe.head()

,susceptible,exposed,infected,recovered,dead
0,8.383243e+07,1.000000,0.000000,0.000000,0.000000
1,8.383243e+07,0.902698,0.171454,0.014369,0.001283
2,8.383243e+07,0.961846,0.313322,0.053320,0.004761
3,8.383243e+07,1.136859,0.454655,0.114618,0.010234
4,8.383243e+07,1.416002,0.615357,0.199848,0.017844


In [10]:
seird_dataframe["total"] = seird_dataframe.sum(axis=1)

## Vary R0 as a function of time t

In [11]:
def deriv1(y, t, N, beta, gamma, delta, alpha, rho):
    S, E, I, R, D = y
    dSdt = -beta(t) * S * I / N
    dEdt = beta(t) * S * I / N - delta * E
    dIdt = delta * E - (1 - alpha) * gamma * I - alpha * rho * I
    dRdt = (1 - alpha) * gamma * I
    dDdt = alpha * rho * I
    return dSdt, dEdt, dIdt, dRdt, dDdt

In [12]:
# Determine how a lockdown after 50 days effects the future spread of the disease
L = 50

In [13]:
N1 = 83832432
D1 = 4.0 # infections lasts four days
gamma1 = 1.0 / D1
delta1 = 1.0 / 5.0  # incubation period of five days
def R_0(t):
    return 5.0 if t < L else 0.9
def beta(t):
    return R_0(t) * gamma1

alpha1 = 0.2  # 20% death rate
rho1 = 1/9  # 9 days from infection until death
S01, E01, I01, R01, D01 = N1-1, 1, 0, 0, 0  # initial conditions: one exposed

In [14]:
t1 = np.linspace(0, 200, 201) # Grid of time points (in days)
y01 = S01, E01, I01, R01, D01 # Initial conditions vector

# Integrate the SIR equations over the time grid, t.
ret1 = odeint(deriv1, y01, t1, args=(N1, beta, gamma1, delta1, alpha1, rho1))
S1, E1, I1, R1, D1 = ret1.T

In [15]:
seird_dataframe_r0 = pd.DataFrame({'susceptible':S1,
                                        'exposed' : E1,
                                        'infected': I1,
                                        'recovered': R1,
                                         'dead': D1
                                         })

seird_dataframe_r0.head()

,susceptible,exposed,infected,recovered,dead
0,8.383243e+07,1.000000,0.000000,0.000000,0.000000
1,8.383243e+07,0.922449,0.168772,0.017751,0.001972
2,8.383243e+07,1.028938,0.308203,0.065606,0.007290
3,8.383243e+07,1.274244,0.452179,0.141358,0.015706
4,8.383243e+07,1.652396,0.623690,0.248319,0.027591


In [16]:
graph_types_1=['infected',
              'susceptible',
              'recovered',
             'exposed',
            'dead'
             ] 

## Dynamic parameters in SIR (infection rate)

In [17]:
df_population=pd.read_csv('../data/processed/pouplation_country.csv',sep=',')  
df_population.dropna(inplace=True)
df_population.set_index('country', inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/pouplation_country.csv'

In [ ]:
timeSteps = np.arange(len(df_analyse['Germany'][35:]))
ydata=np.array(df_analyse['Germany'][35:])
N_init=1000000
# ensure re-initialization 
I_init=ydata[0]
S_init=N_init-I_init
R_init=0

In [ ]:
print(ydata)

In [ ]:
def SIR_model_t(SIR,timeSteps,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    
#     if country in df_population.index:
#     N0=df_population.loc['Germany']['population']
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N_init          #S*I is the 
    dI_dt=beta*S*I/N_init-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

In [ ]:
def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
   
    return integrate.odeint(SIR_model_t, (S_init, I_init, R_init), timeSteps, args=(beta, gamma))[:,1] # we only would like to get dI

In [ ]:
def getOptimalBetaAndGamma(beta,gamma):    

    print(beta)
    print(gamma)
    popt=[beta,gamma]
    fit_odeint(timeSteps, *popt)
    popt, pcov = optimize.curve_fit(fit_odeint, timeSteps, ydata)
    perr = np.sqrt(np.diag(pcov))
     

    # get the final fitted curve
    fitted=fit_odeint(timeSteps, *popt) 
    return popt[0],popt[1]

In [ ]:
b1,g1 = getOptimalBetaAndGamma(0.4,0.1)

In [ ]:
b1,g1

In [ ]:
def SIR_model(SIR,beta,gamma,N):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''

    
    S,I,R=SIR
    dS_dt=-beta*S*I/N       #S*I is the 
    dI_dt=beta*S*I/N-gamma*I
    dR_dt=gamma*I
    
    return([dS_dt,dI_dt,dR_dt])


In [ ]:
def fitCountryCurve(beta,gamma,country,infection_days,measures_days,hold,relax):

    t_initial=infection_days
    t_intro_measures=measures_days
    t_hold=hold
    t_relax=relax
    N0=1000000
    
    beta_max=beta
    beta_min=0.11
    gamma=gamma
    pd_dynamic=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                        np.linspace(beta_min,beta_max,t_relax),
                       ))

    # condition I0+S0+R0=N0
    

    if country in df_population.index:
        N0=df_population.loc[country]['population']
#         N0=1382604367
    I0=df_analyse[country][infection_days]
    S0=N0-I0
    R0=0
    
    print(N0)
        
#     SIRN=np.array([S0,I0,R0,N0])
    SIR=np.array([S0,I0,R0])
    
#     print(S0)
#     print(I0)
#     print(R0)
    
    propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})



    for each_beta in pd_dynamic:
   
        new_delta_vec=SIR_model(SIR,each_beta,gamma,N0)

        SIR=SIR+new_delta_vec

        propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                    'infected':SIR[1],
                                                    'recovered':SIR[2]}, ignore_index=True)
    
    return propagation_rates

In [ ]:
propagation_rates=fitCountryCurve(0.2,0.1,'India',60,70,80,80)
propagation_rates.infected

In [ ]:
fig, ax1 = plt.subplots(1, 1)

ax1.plot(propagation_rates.index,propagation_rates.infected,label='infected',linewidth=3)

## Dashboard

In [ ]:
import plotly.graph_objects as go
import requests
import json
import plotly

In [ ]:
graph_types_dead=['infected',
              'susceptible',
              'recovered',
             'exposed',
            'dead',
            'total'
             ] 

In [ ]:
colors = {
    'background': '#111111',
    'text': '#111111'
}

In [ ]:
country_list=df_analyse.columns[1:]
country_dropdown=[]
for country in country_list:
    country_dropdown.append({'label':country,'value':country})

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html


fig=go.Figure()
fig_1=go.Figure()

app = dash.Dash(external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

app.layout = html.Div([
    
       
                        html.H1(
                            children='SIR Modelling Dashboard',
                            style={
                                'textAlign': 'center',
                                'color': 'white',
                                'fontSize': 36,
                                "border":"2px black solid",
                                'margin-bottom': '30px',
                                'background': '#fb6107'
                            }
                        ),

                  
    
                        html.Div([                             
                                html.H2('SEIRD Modelling for Covid-19',
                                        style={
                                                'textAlign': 'center',
                                                'color': colors['text'],
                                            'text-decoration': 'underline',
                                            'margin-bottom': '30px'
                                }),  
                               
                            html.Div(
                                html.Label('Category:', style={'fontSize':18}), style={'margin-bottom': '10px'}),
                                dcc.Dropdown(
                                    id='sir model',
                                    options=[
                                        {'label': 'infected', 'value': 'infected'},
                                        {'label': 'susceptible', 'value': 'susceptible'},
                                        {'label': 'recovered', 'value': 'recovered'},
                                        {'label': 'exposed', 'value': 'exposed'},
                                        {'label': 'dead', 'value': 'dead'},
                                        {'label': 'total', 'value': 'total'}
                                    ],
                                    value=['infected', 'recovered'], # which are pre-selected        
                                    multi = True
                                ),

                                dcc.Graph(figure=fig, id='main_window_slope')

                                ], style={'margin-bottom': '60px'}), 
    
    
    
                           html.Div([                             
                                html.H2('SEIRD Modelling for Covid-19 with varying R0(t)',
                                        style={
                                                'textAlign': 'center',
                                                'color': colors['text'],
                                            'text-decoration': 'underline',
                                            'margin-bottom': '30px'
                                }),  
                               
                            html.Div(
                                html.Label('Lockdown days = 50:', style={'fontSize':18}), style={'margin-bottom': '10px'}),
                                dcc.Dropdown(
                                    id='sir model_1',
                                    options=[
                                        {'label': 'infected', 'value': 'infected'},
                                        {'label': 'susceptible', 'value': 'susceptible'},
                                        {'label': 'recovered', 'value': 'recovered'},
                                        {'label': 'exposed', 'value': 'exposed'},
                                        {'label': 'dead', 'value': 'dead'}                                        
                                    ],
                                    value=['infected', 'exposed'], # which are pre-selected        
                                    multi = True
                                ),

                                dcc.Graph(figure=fig_1, id='main_window_slope_1')

                                ], style={'margin-bottom': '60px'}), 
    
                        html.H2(children = 'Dynamic parameters in SIR (infection rate)', style={
                            'textAlign': 'center',
                            'color': colors['text'],
                            'text-decoration': 'underline',
                            'margin-bottom': '30px'
                        }),
    
                   html.Div([ 
                        html.Div(
                            html.Label('Select a Country:', style={'fontSize':28}), style={'margin-bottom': '10px'}),
                        dcc.Dropdown(
                                        id='country_drop_down',                                      
                                        options=country_dropdown,
                                        value='Germany', # which are pre-selected
                                        multi=False
                                    ),          
                        
                            ]),
                    
                   html.Br(),
                   html.Div([
                            html.Div([
                                html.Div([
                                        html.Label('Select value for beta',  style={
                                            'textAlign': 'left',
                                            'color': colors['text'], 'fontSize': 18
                                        })]), 

                                   html.Div(
                                   dcc.Slider(
                                        id='slider1',  
                                        max=2,
                                        min=0.2,
                                        value=0.3,
                                        step=0.01,
                                        updatemode='drag',

                                    ),style={"width": "30%", 'margin-top': '10px'} ),
                            ], className="six columns"),

                            html.Div([

                                    html.Div([ html.Label('Select value for gamma' ,  style={
                                            'textAlign': 'left',
                                            'color': colors['text'], 'fontSize': 18
                                        })]),

                                    html.Div(
                                    dcc.Slider(
                                        id='slider2',
                                        max=1,
                                        value=0.2,
                                        step=0.01,
                                        updatemode='drag'       
                                    ),   
                                    style={"width": "30%", 'margin-top': '10px'}),
                            ], className="six columns")
                   
                ],className="row"),
                
    
                html.Div(id='updatemode-output-container'), 
                
                html.Br(),
    
                html.Div([
                    html.Div([
                           html.Label('Infection started after (in days)',  style={
                                            'color': colors['text'], 'fontSize': 18
                                        }),                        
                        dcc.Input(id="infection_start", type="number", value=28)
                    ],className="six columns"),
                    
                    html.Div([
                        html.Label("Measures taken after (in days)",  style={
                                            'color': colors['text'], 'fontSize': 18
                                        }),                           
                        dcc.Input(id="measures_start", type="number", value=14), 
                    ],className="six columns"),
                    
                ], className="row"),
              html.Br(),
    
              html.Div([
                    
                html.Div([
                    html.Label('Hold (in days)',  style={
                                            'color': colors['text'], 'fontSize': 18
                                        }),                      
                    dcc.Input(id="hold_start", type="number", value=60)
                ],className="six columns"),
                                     
                html.Div([
                    html.Label('Relax (in days)',  style={
                                            'color': colors['text'], 'fontSize': 18
                                        }),                     
                    dcc.Input(id="relax_start", type="number", value=60),
                    
                ],className="six columns"),
                
            ],className="row"),
          
           html.Br(),
           html.Div([
               html.Label("Click to simulate SIR curve",style={
                                            'color': colors['text'], 'fontSize': 18
                                        }),
               html.Br(),
               html.Button('Submit', id='submit_btn', n_clicks=0)
           ]), 
           
    html.Div([dcc.Graph(id='my-output')]),


            ],style = {'margin':'auto','width': "50%"})  



In [ ]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('sir model', 'value')])
def update_figure(graph_types):
    
    traces = [] 
    for each in graph_types:
        traces.append(dict(x=seird_dataframe.index,
                                y=seird_dataframe[each],
                                mode='lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name=each
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=900,
                height=700,
                align= "center",
                title = "SIR Modelling with Exposed and Dead Components",
                #                 xaxis_title="Time",
#                 yaxis_title="Confirmed infected people (source johns hopkins csse, log-scale)",
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="black"),
                       "title": "Time in days" ,
                        'titlefont': dict(
                        family = 'Comic Sans MS',
                        size = 18, 
                        color = '#e36414')
                      },
                
                yaxis={
                    "type": "log",
                    "title": "Number of people",
                    'tickfont':dict(size=14,color="black"),
                     'titlefont': dict(
                        family = 'Comic Sans MS',
                        size = 18, 
                        color = '#e36414')
                      }
        )
    }

In [ ]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope_1', 'figure'),
    [Input('sir model_1', 'value')])
def update_figure(graph_types):
    
    traces = [] 
    for each in graph_types:
        traces.append(dict(x=seird_dataframe_r0.index,
                                y=seird_dataframe_r0[each],
                                mode='lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name=each
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=900,
                height=700,
                align= "center",
                title = "SIR Modelling with time dependent R0",
                #                 xaxis_title="Time",
#                 yaxis_title="Confirmed infected people (source johns hopkins csse, log-scale)",
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="black"),
                       "title": "Time in days" ,
                        'titlefont': dict(
                        family = 'Comic Sans MS',
                        size = 18, 
                        color = '#e36414')
                      },
                
                yaxis={
                    "title": "Number of people",
                    'tickfont':dict(size=14,color="black"),
                     'titlefont': dict(
                        family = 'Comic Sans MS',
                        size = 18, 
                        color = '#e36414')
                      }
        )
    }

In [ ]:
@app.callback(
    Output('updatemode-output-container', 'children'),
    [Input('slider1', 'value'),
    Input('slider2', 'value')]
)
def update_slider(value1, value2):
    return 'Beta Value: {} | \
            Gamma Value: {}'.format(value1, value2)


In [ ]:
@app.callback(
    dash.dependencies.Output('my-output', 'figure'),
    [
        dash.dependencies.Input('country_drop_down', 'value'),
        dash.dependencies.Input('submit_btn','n_clicks')],
    [        
        dash.dependencies.State('infection_start', 'value'),
        dash.dependencies.State('measures_start', 'value'),
        dash.dependencies.State('hold_start', 'value'),
        dash.dependencies.State('relax_start', 'value'),
        dash.dependencies.State('slider1', 'value'),
        dash.dependencies.State('slider2', 'value')
    ])

def update_output(country_name,n_clicks,infection_days,measures_days,hold,relax,beta,gamma):
    traces=[]
    t=np.arange(len(df_analyse[country_name][infection_days:]))
    df_dynamic = fitCountryCurve(beta,gamma,country_name,infection_days,measures_days,hold,relax)
    df_test = pd.DataFrame({'t' : t, 'ydata' : df_analyse[country_name][infection_days:]})
    traces.append(dict(x=df_test.t,
                                y=df_test.ydata,
                                marker = {
                                    "color": "rgb(255, 51, 0)"},
                                type='bar',
                                name='Confirmed Infected cases')),
    traces.append(dict(x=df_dynamic.index,
                                y=df_dynamic.infected,
                                mode='lines',
                                marker = {
                                    "color": "rgb(0, 0, 255)"},
                                opacity=0.9,
                                line_width=4,
                                marker_size=4, 
                                name='Modelled Infected cases'
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=900,
                height=700,     
                title = "Fit of SIR model for " + country_name + " cases with given parameters.",
                #                 xaxis_title="Time",
            #                 yaxis_title="Confirmed infected people (source johns hopkins csse, log-scale)",
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="black"),
                       "title": "Days" ,
                         'titlefont': dict(
                            family = 'Comic Sans MS',
                            size = 18, 
                            color = '#e36414')

                      },
                yaxis={
                    'range':'[1.1,5.5]',
                    'title':"No. of cases",
                    'type':'log',
                    "title": "Population infected",
                       'tickfont':dict(size=14,color="black"),
                       'titlefont': dict(
                            family = 'Comic Sans MS',
                            size = 18, 
                            color = '#e36414')
                      }
            )
         }   

In [ ]:
# from dash.dependencies import Input, Output, State


# @app.callback(
#     Output('my-output', 'figure'),
# #     [Input('textarea-state-example-button', 'n_clicks')],
#     [Input('slider1', 'value'),
#     Input('slider2', 'value'),
#     Input('country_drop_down', 'value')])
     
# def update_output(value1,value2,country_name):
#     traces = [] 
# #     if n_clicks > 0:
#     popt=[value1, value2]
# #     fitted=fit_odeint(t, *popt)
#     t=np.arange(len(df_analyse[country_name]))
#     df_dynamic = fitCountryCurve(value1,value2,country_name)
#     df_test = pd.DataFrame({'t' : t, 'ydata' : df_analyse[country_name]})
#     traces.append(dict(x=df_test.t,
#                             y=df_test.ydata,
#                             mode='lines',
#                             opacity=0.9,
#                             line_width=2,
#                             marker_size=4, 
#                             name='Y-data'
#                     )),
#     traces.append(dict(x=df_dynamic.index,
#                             y=df_dynamic.infected,
#                             mode='lines',
#                             opacity=0.9,
#                             line_width=2,
#                             marker_size=4, 
#                             name='Fitted curve'
#                     )
#             )

#     return {
#         'data': traces,
#         'layout': dict (
#             width=900,
#             height=700,     
#             title = "Fit of SIR model for Germany cases with different values of beta and gamma.",
#             #                 xaxis_title="Time",
#         #                 yaxis_title="Confirmed infected people (source johns hopkins csse, log-scale)",
#             xaxis={'tickangle':-45,
#                     'nticks':20,
#                     'tickfont':dict(size=14,color="black"),
#                    "title": "Days" ,
#                      'titlefont': dict(
#                         family = 'Comic Sans MS',
#                         size = 18, 
#                         color = '#e36414')

#                   },
#             yaxis={"title": "Population infected",
#                    'tickfont':dict(size=14,color="black"),
#                    'titlefont': dict(
#                         family = 'Comic Sans MS',
#                         size = 18, 
#                         color = '#e36414')
#                   }
#         )
#      }   

In [ ]:
app.run_server(debug=True, use_reloader=False)